In [1]:
import pandas as pd

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pickle

from torchsummary import summary

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

import time

In [ ]:
# !pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
annotated_data = pd.read_csv('incomplete_annotations_data2.csv')

# annotated_data = full_data[full_data['Subjectivity'].notnull()]
# unannotated_data = full_data[full_data['Subjectivity'].isnull()]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Preprocessing Data

In [ ]:
annotated_data[annotated_data['Comment'].isnull()]

,Brand,Search Term,Comment,Source,Metadata,Subjectivity,Polarity,Subjectivity 2,Polarity 2
2590,JW Anderson,JW Anderson,NaN,Instagram,{'Likes_and_timestamp': '0 likes on 2023-11-20...,0.0,NaN,0.0,NaN


In [ ]:
annotated_data = annotated_data.dropna(subset=['Comment'])
print(annotated_data.isnull().sum())

Brand              103
Search Term        174
Comment              0
Source               0
Metadata           234
Subjectivity         0
Polarity          1122
Subjectivity 2       0
Polarity 2         794
dtype: int64


In [ ]:
with open('abbreviations_list.pkl', 'rb') as file:
    abbreviations = pickle.load(file)

print(abbreviations)

{"ain't": 'is not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd": 'i would', "i'd've": 'i would have', "i'll": 'i will', "i'll've": 'i will have', "i'm": 'i am', "i've": 'i have', "isn't": 'is not', "it'd": 'it would', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'migh

In [ ]:
# Creating extra column for preprocessed text
annotated_data['Preprocessed Comment'] = annotated_data['Comment']

In [ ]:
# Normalizing emojis

import emoji

def demojize_with_delimiters(text):
    return emoji.demojize(text, delimiters=(" ", " "))

annotated_data['Preprocessed Comment'] = annotated_data['Preprocessed Comment'].apply(lambda x: demojize_with_delimiters(x) if isinstance(x, str) else x)

In [ ]:
# Lowercasing

annotated_data['Preprocessed Comment'] = annotated_data['Preprocessed Comment'].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
# Removing stopwords
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    # Ensure the input is a string
    if isinstance(text, str):
        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Get the list of stopwords
        stop_words = set(stopwords.words('english'))

        # Remove stopwords from the tokenized words
        filtered_words = [word for word in words if word.lower() not in stop_words]

        # Join the filtered words back into a single string
        filtered_text = ' '.join(filtered_words)

        return filtered_text
    else:
        return text

annotated_data['Preprocessed Comment'] = annotated_data['Preprocessed Comment'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to manually tokenize text including punctuations
def custom_tokenize(text):
    # Regex pattern to match words (including contractions) and separate punctuation
    tokens = re.findall(r"[\w']+|[.,!?;]", text)
    return tokens

# Normalize slangs and abbreviations
def normalize_slangs_abbreviations_custom(text, slang_dict):
    if isinstance(text, str):
        tokens = custom_tokenize(text)
        normalized_tokens = [slang_dict.get(token.lower(), token) for token in tokens]
        # Reconstruct the text
        normalized_text = ' '.join(normalized_tokens).replace(" ,", ",").replace(" .", ".").replace(" !", "!").replace(" ?", "?")
        return normalized_text
    else:
        return text

annotated_data['Preprocessed Comment'] = annotated_data['Preprocessed Comment'].apply(lambda x: normalize_slangs_abbreviations_custom(x, abbreviations))

In [ ]:
print(annotated_data['Comment'].iloc[20])
print(annotated_data['Preprocessed Comment'].iloc[20])

WHY is Hermes even getting involved at the Lotus casino, seems like a damn waste of time – tho I know they're probably trying to give Luke more backstory before the finale
hermes even getting involved lotus casino, seems like damn waste time though know 're probably trying give luke backstory finale


# BERT

## Subjectivity Detection

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

annotated_texts = annotated_data['Preprocessed Comment'].tolist()
annotated_labels = annotated_data['Subjectivity'].tolist()

# Split the data into train and validation sets, ensuring equal distribution of labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_index, val_index = next(sss.split(annotated_texts, annotated_labels))
train_texts = [annotated_texts[i] for i in train_index]
val_texts = [annotated_texts[i] for i in val_index]
train_labels = [annotated_labels[i] for i in train_index]
val_labels = [annotated_labels[i] for i in val_index]

# Tokenize texts using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_data_train = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
encoded_data_val = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')

# Extract attention masks
attention_masks_train = encoded_data_train['attention_mask']
attention_masks_val = encoded_data_val['attention_mask']

# Convert labels to tensor
labels_train = torch.tensor(train_labels)
labels_val = torch.tensor(val_labels)

# Define DataLoader for training and validation sets
train_dataset = TensorDataset(encoded_data_train['input_ids'], attention_masks_train, labels_train)
val_dataset = TensorDataset(encoded_data_val['input_ids'], attention_masks_val, labels_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [2]:
# Initialize BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                       num_labels=2,
                                                       hidden_dropout_prob=0.2,
                                                       attention_probs_dropout_prob=0.2)

summary(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                   Param #
├─BertModel: 1-1                         --
|    └─BertEmbeddings: 2-1               --
|    |    └─Embedding: 3-1               23,440,896
|    |    └─Embedding: 3-2               393,216
|    |    └─Embedding: 3-3               1,536
|    |    └─LayerNorm: 3-4               1,536
|    |    └─Dropout: 3-5                 --
|    └─BertEncoder: 2-2                  --
|    |    └─ModuleList: 3-6              85,054,464
|    └─BertPooler: 2-3                   --
|    |    └─Linear: 3-7                  590,592
|    |    └─Tanh: 3-8                    --
├─Dropout: 1-2                           --
├─Linear: 1-3                            1,538
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─BertModel: 1-1                         --
|    └─BertEmbeddings: 2-1               --
|    |    └─Embedding: 3-1               23,440,896
|    |    └─Embedding: 3-2               393,216
|    |    └─Embedding: 3-3               1,536
|    |    └─LayerNorm: 3-4               1,536
|    |    └─Dropout: 3-5                 --
|    └─BertEncoder: 2-2                  --
|    |    └─ModuleList: 3-6              85,054,464
|    └─BertPooler: 2-3                   --
|    |    └─Linear: 3-7                  590,592
|    |    └─Tanh: 3-8                    --
├─Dropout: 1-2                           --
├─Linear: 1-3                            1,538
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0

In [ ]:
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

best_val_loss = float('inf')  # Initialize with positive infinity
best_val_accuracy = 0.0
best_epoch = 0
patience = 5  # Number of epochs to wait for improvement

no_improvement_count = 0

train_start_time = time.time()

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    epoch_start_time = time.time()

    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)  # No need to pass labels here
        logits = outputs.logits

        labels = labels.long()

        loss = F.cross_entropy(logits, labels)  # Compute cross-entropy loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)  # No need to pass labels during evaluation
            logits = outputs.logits

            labels = labels.long()

            val_loss += F.cross_entropy(logits, labels).item()

            val_preds.extend(torch.argmax(logits, dim=1).tolist())
            val_targets.extend(labels.tolist())

    val_loss /= len(val_loader)
    val_accuracy = sum(1 for p, t in zip(val_preds, val_targets) if p == t) / len(val_preds)

    precision = precision_score(val_targets, val_preds)
    recall = recall_score(val_targets, val_preds)
    f1 = f1_score(val_targets, val_preds)

    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val Precision: {precision}, Val Recall: {recall}, Val F1: {f1}, Time taken per epoch: {epoch_end_time-epoch_start_time:.2f}")

    # Update best validation loss and accuracy
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_accuracy = val_accuracy
        best_val_precision = precision
        best_val_recall = recall
        best_val_f1 = f1

        best_epoch_loss = epoch + 1
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f"No improvement for {patience} epochs. Early stopping...")
        break

train_end_time = time.time()

print(f"Best Validation Loss: {best_val_loss} at Epoch {best_epoch_loss}")
print(f"Best Validation Accuracy: {best_val_accuracy} at Epoch {best_epoch_loss}")
print(f"Best Validation Precision: {best_val_precision} at Epoch {best_epoch_loss}")
print(f"Best Validation Recall: {best_val_recall} at Epoch {best_epoch_loss}")
print(f"Best Validation F1: {best_val_f1} at Epoch {best_epoch_loss}")

print(f"Time taken to train the model: {train_end_time - train_start_time:.2f} seconds")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 118/118 [02:46<00:00,  1.41s/it]


Epoch 1: Train Loss: 73.50101447105408, Val Loss: 0.6125095515858894, Val Accuracy: 0.6546583850931676, Val Precision: 0.8544776119402985, Val Recall: 0.4893162393162393, Val F1: 0.6222826086956522, Time taken per epoch: 194.38


Epoch 2: 100%|██████████| 118/118 [02:52<00:00,  1.46s/it]


Epoch 2: Train Loss: 62.46815751492977, Val Loss: 0.49883832709462034, Val Accuracy: 0.7490683229813665, Val Precision: 0.7736625514403292, Val Recall: 0.8034188034188035, Val F1: 0.7882599580712789, Time taken per epoch: 200.10


...

Epoch 5: Train Loss: 25.421000864356756, Val Loss: 0.6835929117366379, Val Accuracy: 0.7614906832298136, Val Precision: 0.7664092664092664, Val Recall: 0.8482905982905983, Val F1: 0.8052738336713996, Time taken per epoch: 200.16


Epoch 6: 100%|██████████| 118/118 [02:52<00:00,  1.46s/it]


Epoch 6: Train Loss: 18.62071276968345, Val Loss: 0.6244448452603584, Val Accuracy: 0.7590062111801242, Val Precision: 0.7686274509803922, Val Recall: 0.8376068376068376, Val F1: 0.801635991820041, Time taken per epoch: 200.30


Epoch 7: 100%|██████████| 118/118 [02:52<00:00,  1.46s/it]


Epoch 7: Train Loss: 16.773903043940663, Val Loss: 0.7709790102260954, Val Accuracy: 0.7453416149068323, Val Precision: 0.7827956989247312, Val Recall: 0.7777777777777778, Val F1: 0.7802786709539122, Time taken per epoch: 200.11
No improvement for 5 epochs. Early stopping...
Best Validation Loss: 0.49883832709462034 at Epoch 2
Best Validation Accuracy: 0.7490683229813665 at Epoch 2
Best Validation Precision: 0.7736625514403292 at Epoch 2
Best Validation Recall: 0.8034188034188035 at Epoch 2
Best Validation F1: 0.7882599580712789 at Epoch 2
Time taken to train the model: 1395.22 seconds


Around 1.4s per classification of record

## Polarity Detection

In [ ]:
# Preprocess the annotated data (assuming it has columns 'text' and 'polarity')
annotated_polarity_data = annotated_data[annotated_data['Subjectivity']==1]

annotated_texts = annotated_polarity_data['Preprocessed Comment'].tolist()
annotated_labels = annotated_polarity_data['Polarity'].tolist()

# Split the data into train and validation sets, ensuring equal distribution of labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_index, val_index = next(sss.split(annotated_texts, annotated_labels))
train_texts = [annotated_texts[i] for i in train_index]
val_texts = [annotated_texts[i] for i in val_index]
train_labels = [annotated_labels[i] for i in train_index]
val_labels = [annotated_labels[i] for i in val_index]

# Tokenize texts using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_data_train = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
encoded_data_val = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')

# Extract attention masks
attention_masks_train = encoded_data_train['attention_mask']
attention_masks_val = encoded_data_val['attention_mask']

# Convert labels to tensor
labels_train = torch.tensor(train_labels)
labels_val = torch.tensor(val_labels)

# Define DataLoader for training and validation sets
train_dataset = TensorDataset(encoded_data_train['input_ids'], attention_masks_train, labels_train)
val_dataset = TensorDataset(encoded_data_val['input_ids'], attention_masks_val, labels_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Initialize BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                       num_labels=2,
                                                       hidden_dropout_prob=0.2,
                                                       attention_probs_dropout_prob=0.2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

best_val_loss = float('inf')  # Initialize with positive infinity
best_val_accuracy = 0.0
best_epoch = 0
patience = 5  # Number of epochs to wait for improvement

no_improvement_count = 0

train_start_time = time.time()

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    epoch_start_time = time.time()

    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)  # No need to pass labels here
        logits = outputs.logits

        labels = labels.long()

        loss = F.cross_entropy(logits, labels)  # Compute cross-entropy loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)  # No need to pass labels during evaluation
            logits = outputs.logits

            labels = labels.long()

            val_loss += F.cross_entropy(logits, labels).item()

            val_preds.extend(torch.argmax(logits, dim=1).tolist())
            val_targets.extend(labels.tolist())

    val_loss /= len(val_loader)
    val_accuracy = sum(1 for p, t in zip(val_preds, val_targets) if p == t) / len(val_preds)

    precision = precision_score(val_targets, val_preds)
    recall = recall_score(val_targets, val_preds)
    f1 = f1_score(val_targets, val_preds)

    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val Precision: {precision}, Val Recall: {recall}, Val F1: {f1}, Time taken: {epoch_end_time-epoch_start_time:.2f}")

    # Update best validation loss and accuracy
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_accuracy = val_accuracy
        best_val_precision = precision
        best_val_recall = recall
        best_val_f1 = f1

        best_epoch_loss = epoch + 1
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f"No improvement for {patience} epochs. Early stopping...")
        break

train_end_time = time.time()

print(f"Best Validation Loss: {best_val_loss} at Epoch {best_epoch_loss}")
print(f"Best Validation Accuracy: {best_val_accuracy} at Epoch {best_epoch_loss}")
print(f"Best Validation Precision: {best_val_precision} at Epoch {best_epoch_loss}")
print(f"Best Validation Recall: {best_val_recall} at Epoch {best_epoch_loss}")
print(f"Best Validation F1: {best_val_f1} at Epoch {best_epoch_loss}")

print(f"Time taken to train the model: {train_end_time - train_start_time:.2f} seconds")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 69/69 [01:39<00:00,  1.45s/it]


Epoch 1: Train Loss: 42.72779059410095, Val Loss: 0.5172286242246628, Val Accuracy: 0.7585470085470085, Val Precision: 0.79296875, Val Recall: 0.7718631178707225, Val F1: 0.7822736030828517, Time taken: 116.22


Epoch 2: 100%|██████████| 69/69 [01:39<00:00,  1.45s/it]


Epoch 2: Train Loss: 32.396301835775375, Val Loss: 0.5239216481645902, Val Accuracy: 0.7905982905982906, Val Precision: 0.8260869565217391, Val Recall: 0.7946768060836502, Val F1: 0.8100775193798451, Time taken: 116.11


Epoch 3: 100%|██████████| 69/69 [01:39<00:00,  1.44s/it]


...

Epoch 4: Train Loss: 14.469338877126575, Val Loss: 0.9173563969631989, Val Accuracy: 0.7564102564102564, Val Precision: 0.8634146341463415, Val Recall: 0.6730038022813688, Val F1: 0.7564102564102564, Time taken: 115.98


Epoch 5: 100%|██████████| 69/69 [01:39<00:00,  1.45s/it]


Epoch 5: Train Loss: 13.335239961743355, Val Loss: 0.8938806588451068, Val Accuracy: 0.7564102564102564, Val Precision: 0.8941798941798942, Val Recall: 0.6425855513307985, Val F1: 0.7477876106194691, Time taken: 116.06


Epoch 6: 100%|██████████| 69/69 [01:39<00:00,  1.45s/it]


Epoch 6: Train Loss: 9.274033974390477, Val Loss: 0.8694699347019196, Val Accuracy: 0.7692307692307693, Val Precision: 0.8506787330316742, Val Recall: 0.714828897338403, Val F1: 0.7768595041322314, Time taken: 116.03
No improvement for 5 epochs. Early stopping...
Best Validation Loss: 0.5172286242246628 at Epoch 1
Best Validation Accuracy: 0.7585470085470085 at Epoch 1
Best Validation Precision: 0.79296875 at Epoch 1
Best Validation Recall: 0.7718631178707225 at Epoch 1
Best Validation F1: 0.7822736030828517 at Epoch 1
Time taken to train the model: 696.32 seconds
